In [2]:
 %%sh
pip install -q --upgrade pip
pip install -q --upgrade boto3 botocore
pip install -q tqdm nibabel pydicom numpy pylibjpeg-openjpeg #pathlib2 monai 

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.31.25 which is incompatible.
awscli 1.29.14 requires

In [4]:
import pydicom
from pydicom.sequence import Sequence
from pydicom import Dataset , DataElement 
from pydicom.dataset import FileDataset, FileMetaDataset
from pydicom.uid import UID
from pydicom.pixel_data_handlers.util import convert_color_space , apply_color_lut
from openjpeg import decode
import array
import json
import logging
import importlib  
import boto3
import sagemaker
from sagemaker import get_execution_role
import io
import sys
import time
import os
import pandas as pd
from botocore.exceptions import ClientError
logging.basicConfig( level="INFO" )
# logging.basicConfig( level="DEBUG" )
from src.Api import MedicalImaging 
medicalimaging = MedicalImaging()

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name
bucket = sagemaker.Session().default_bucket()
role = get_execution_role()
print(f"S3 Bucket is {bucket}")
print(f"IAM role is {role}")

/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


S3 Bucket is sagemaker-us-east-1-659535263284
IAM role is arn:aws:iam::659535263284:role/service-role/AmazonSageMaker-ExecutionRole-20230313T165490


In [5]:
InputBucketName = "" ## store your coherent dicom images here

In [6]:
DatastoreName = "CoherentDataStoreWorkshop"
datastoreList = medicalimaging.listDatastores()

res_createstore = None
for datastore in datastoreList["datastoreSummaries"]:
    if datastore["datastoreName"] == DatastoreName:
        res_createstore = datastore
        break
if res_createstore is None:        
    res_createstore = medicalimaging.createDatastore(DatastoreName)

datastoreId = res_createstore['datastoreId']
res_getstore = medicalimaging.getDatastore(res_createstore['datastoreId'])    
status = res_getstore['datastoreProperties']['datastoreStatus']
while status!='ACTIVE':
    time.sleep(30)
    res_getstore = medicalimaging.getDatastore(res_createstore['datastoreId'])    
    status = res_getstore['datastoreProperties']['datastoreStatus']
    print(status)
print(f"datastoreId: {datastoreId}; status: {status}")

INFO:root:List Datastores  : 139.53733444213867 ms
INFO:root:Get Datastore  : 90.18421173095703 ms


datastoreId: 538dd671d2b44906a90edc23edd94757; status: ACTIVE


In [ ]:
res_startimportjob = medicalimaging.startImportJob(
    res_createstore['datastoreId'],
    role,
    's3://'+InputBucketName+'/unzipped/dicom/', 
    's3://'+bucket+'/ahi_importjob_output/'
)

jobId = res_startimportjob['jobId']
jobstatus = medicalimaging.getImportJob(datastoreId, jobId)['jobProperties']['jobStatus']
while jobstatus!='COMPLETED':
    time.sleep(30)
    jobstatus = medicalimaging.getImportJob(datastoreId, jobId)['jobProperties']['jobStatus']
print(f"jobstatus is {jobstatus}")

INFO:root:Start Import Job  : 428.2948970794678 ms
INFO:root:Get Import Job  : 107.45453834533691 ms


In [12]:
imageSetIds = {}
s3=boto3.client('s3')
try:
    response = s3.head_object(Bucket=bucket, Key=f"ahi_importjob_output/{datastoreId}-DicomImport-{jobId}/job-output-manifest.json")
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        data = s3.get_object(Bucket=bucket, Key=f"ahi_importjob_output/{datastoreId}-DicomImport-{jobId}/SUCCESS/success.ndjson")
        contents = data['Body'].read().decode("utf-8")
        for l in contents.splitlines():
            isid = json.loads(l)['importResponse']['imageSetId']
            if isid in imageSetIds:
                imageSetIds[isid]+=1
            else:
                imageSetIds[isid]=1
except ClientError:
    pass


print("number of image sets: {}".format(len(imageSetIds)))

number of image sets: 298


In [13]:
%store datastoreId
%store imageSetIds
%store jobId

Stored 'datastoreId' (str)
Stored 'imageSetIds' (dict)
Stored 'jobId' (str)


## (Optional) Save JSON to S3

In [9]:
for s in imageSetIds.keys():
    json_dicom_header = medicalimaging.getMetadata(datastoreId, s)
    patient = json_dicom_header['Patient']['DICOM']
    patient['imagesetid'] = s
    s3.put_object(
        Body=json.dumps(patient),
        Bucket=OutputBucketName,
        Key='dicom_header/json/patient/{}'.format(s)
    )
    study=json_dicom_header['Study']['DICOM']
    study['imagesetid'] = s
    s3.put_object(
        Body=json.dumps(study),
        Bucket=OutputBucketName,
        Key='dicom_header/json/study/{}'.format(s)
    )
    for se in list(json_dicom_header['Study']['Series'].keys()):
        s3.put_object(
            Body=json.dumps(json_dicom_header['Study']['Series'][se]['DICOM']),
            Bucket=OutputBucketName,
            Key='dicom_header/json/series/{}'.format(s)
        )
        for i in list(json_dicom_header['Study']['Series'][se]['Instances']):
            s3.put_object(
                Body=json.dumps(json_dicom_header['Study']['Series'][se]['Instances'][i]),
                Bucket=OutputBucketName,
                Key='dicom_header/json/series/{}'.format(s)
            )

INFO:root:Metadata fetch  : 248.78931045532227 ms
INFO:root:Metadata fetch  : 217.93746948242188 ms
INFO:root:Metadata fetch  : 263.5033130645752 ms
INFO:root:Metadata fetch  : 228.8510799407959 ms
INFO:root:Metadata fetch  : 224.68209266662598 ms
INFO:root:Metadata fetch  : 216.078519821167 ms
INFO:root:Metadata fetch  : 206.32648468017578 ms
INFO:root:Metadata fetch  : 214.79153633117676 ms
INFO:root:Metadata fetch  : 186.0511302947998 ms
INFO:root:Metadata fetch  : 243.12877655029297 ms
INFO:root:Metadata fetch  : 282.14311599731445 ms
INFO:root:Metadata fetch  : 227.65445709228516 ms
INFO:root:Metadata fetch  : 245.194673538208 ms
INFO:root:Metadata fetch  : 211.95077896118164 ms
INFO:root:Metadata fetch  : 225.4354953765869 ms
INFO:root:Metadata fetch  : 195.9846019744873 ms
INFO:root:Metadata fetch  : 213.026762008667 ms
INFO:root:Metadata fetch  : 226.2578010559082 ms
INFO:root:Metadata fetch  : 230.38601875305176 ms
INFO:root:Metadata fetch  : 178.40170860290527 ms
INFO:root:Me

In [5]:
whos

Variable                              Type        Data/Info
-----------------------------------------------------------
DiscoveryWidget                       type        <class '__main__.DiscoveryWidget'>
Loader                                ABCMeta     <class 'importlib.abc.Loader'>
MetaPathFinder                        ABCMeta     <class 'importlib.abc.MetaPathFinder'>
ModuleSpec                            type        <class '_frozen_importlib.ModuleSpec'>
PACKAGE_DIRS                          type        <class '__main__.PACKAGE_DIRS'>
SageMakerDataWranglerPandasFinder     ABCMeta     <class '__main__.SageMake<...>ataWranglerPandasFinder'>
display                               function    <function display at 0x7f89fd75daf0>
has_pandas_finder                     function    <function has_pandas_finder at 0x7f89f85eeb80>
is_sagemaker_datawrangler_installed   function    <function is_sagemaker_da<...>talled at 0x7f89f85eeaf0>
os                                    module      <module 

## Clean Up

In [ ]:
try:
    s3res = boto3.resource('s3')
    bucket = s3res.Bucket(InputBucketName)
    bucket.object_versions.delete()
    s3.delete_bucket(Bucket=InputBucketName)
    bucket = s3res.Bucket(OutputBucketName)
    bucket.object_versions.delete()
    s3.delete_bucket(Bucket=OutputBucketName)
except ClientError  as e:
    if e.response['Error']['Code'] == 'NoSuchBucket':
        print("Bucket already deleted")
    
try: 
    resp = iam.detach_role_policy(PolicyArn=respons_createpolicy['Policy']['Arn'],RoleName=response_createrole['Role']['RoleName'])
    resp = iam.delete_policy(PolicyArn=respons_createpolicy['Policy']['Arn'])
    resp = iam.delete_role(RoleName=response_createrole['Role']['RoleName'])
except ClientError as ee:
    if ee.response['Error']['Code'] == 'NoSuchEntity':
        print("Policy not attached, ignore")
    else: 
        print(ee)